In [2]:
import xlwings as xw
## number to letter dict
import string
from itertools import combinations_with_replacement as cwr
one_n2a = dict(zip(range(1, 27), string.ascii_lowercase))
two_n2a_list = ["".join(comb) for comb in cwr(string.ascii_lowercase, 2)]
two_n2a = dict(zip(range(27, 53), two_n2a_list))
n2a = {**one_n2a,**two_n2a}

## initiate the integrated spec
wb_0 = xw.Book('hk tagging specs.xlsx')
list_0 = wb_0.sheets['list']
spec_0 = wb_0.sheets['spec']

def integrate_specs():
    list_index = 1
    max_index = list_0.range('A1').current_region.last_cell.row - 2
    start_row = 3
    ## define dl_n as dl number in spec_0
    dl_n = len(spec_0.range('B2').expand('right').value)
    ## clear existing values
    spec_0.range('A3:AZ3').expand('down').clear()

    while list_index < max_index + 1:
        ## define datalayer index
        d_index = 0
        ## select input spec file
        wb_1 = xw.Book(list_0.range('C'+str(list_index+2)).value + '.xlsx')
        ## select input spec sheet
        spec_1 = wb_1.sheets['spec']
        ## count input row
        row_1 = spec_1.range('A1').current_region.last_cell.row - 2
        
        ## copy value from spec_1 to spec_0
        while d_index < dl_n:
            ## define dl as first data layer in spec_0
            dl = spec_0.range('B2:AZ2').value[d_index]
            ## define c_index as the column index of dl in spec_1
            c_index = spec_1.range('B2:AZ2').value.index(dl)
            ## define dl_v as dl value in spec_1
            dl_v = spec_1.range('B3:AZ3').expand('down').options(transpose=True).value[c_index]
            ## cell position as cp, right column and starting cells in spec_0
            cp = n2a[d_index+2]+str(start_row)
            ## input dl_v into spec_0
            spec_0.range(cp).options(transpose=True).value = dl_v
            ## go to next column
            d_index += 1
        
        ## add MCTAG key
        spec_0.range('A'+str(start_row)+':A'+str(start_row+row_1-1)).value = list_0.range('D'+str(list_index+2)).value
        ## go to next spec
        wb_1.close()
        start_row = start_row + row_1
        list_index += 1
        
integrate_specs()